Import Lib

In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver

from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

c:\Users\Lenovo\Desktop\text\LangGraph\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
jokeLLM = ChatGoogleGenerativeAI(
  model='gemini-2.0-flash',
  temperature=0.7
)

### 1. State

In [4]:
class jokeState(TypedDict):
  topic:str
  joke:str
  explaination:str

### 2. Function

In [5]:
def jokeGen(state:jokeState)->jokeState:
  prompt = f"""Write a funny, creative, and clean joke based on the topic: {state['topic']}. 
Make sure the joke is easy to understand, engaging, and suitable for a general audience."""

  response = jokeLLM.invoke(prompt).content
  return {'joke':response}

In [6]:
def jokeEx(state:jokeState)->jokeState:
  prompt = prompt = f"""
I have written a joke on the topic: "{state['topic']}".
Here is the joke: "{state['joke']}".

Please explain the joke in simple and clear terms so that anyone,
even someone unfamiliar with the topic, can understand why it is funny. 
Also highlight the wordplay, humor style (pun, irony, sarcasm, etc.), 
and the cultural or contextual background if relevant.
"""

  response = jokeLLM.invoke(prompt).content
  return {'explaination':response}

### 3. Graph

In [7]:
graph = StateGraph(jokeState)

graph.add_node('jokeGenerator', jokeGen)
graph.add_node('jokeExplaination', jokeEx)

graph.add_edge(START, 'jokeGenerator')
graph.add_edge('jokeGenerator', 'jokeExplaination')
graph.add_edge('jokeExplaination', END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

##### 3.1 WorkFlow Representation

In [ ]:
workflow

##### 3.2 WorkFlow Execution

3.2.1 Config 

In [ ]:
config1 = {
  'configurable':{'thread_id':'1'}
}

config12 = {
  'configurable':{'thread_id':'12'}
}

3.2.2 State Invoking

In [10]:
initalState = {
  'topic':'Mango'
}
workflow.invoke(initalState, config=config1)

{'topic': 'Mango',
 'joke': 'Why did the mango break up with the apricot?\n\nBecause it said, "I\'m starting to feel like you\'re taking me for GRANITE!" (As in, taking me for granted, but a play on the word "granite" which sounds like the pit of a mango!)',
 'explaination': 'Okay, here\'s a breakdown of the mango joke:\n\n**Explanation:**\n\nThe joke is about a mango ending a relationship with an apricot. The reason given is that the mango feels unappreciated. Instead of saying the common phrase "taking me for *granted*," it says "taking me for *granite*." This is funny because:\n\n*   **"Granite" sounds like "granted."**\n*   **"Granite" is a type of rock.**\n*   **Mangoes have a hard, stony pit inside them.** This pit is technically called a "stone" or "seed," but the association with something hard and rock-like is there.\n\nSo, the joke implies that the apricot is treating the mango like a hard, unyielding object instead of a valued partner.\n\n**Wordplay:**\n\n*   **Pun (Homophon

In [11]:
initalState = {
  'topic':'Pineapple'
}
workflow.invoke(initalState, config=config12)

{'topic': 'Pineapple',
 'joke': 'Why did the pineapple break up with the coconut?\n\nBecause he said she was too clingy! He needed some space to, you know, be a little more prickly! ',
 'explaination': 'Okay, here\'s the breakdown of the pineapple joke:\n\n**Explanation:**\n\nThe joke is funny because it uses the characteristics of pineapples and coconuts to create a humorous situation about a romantic breakup. It plays on the common stereotype of clingy partners needing space in a relationship.\n\n*   The coconut is described as "clingy" because coconuts grow in clusters on palm trees, appearing to be close together or "clingy".\n*   The pineapple wants space to be "prickly" because pineapples have a tough, spiky exterior. He wants to embrace his natural characteristic, which implies he wants to be more independent and less easy to approach.\n\n**Wordplay/Humor Style:**\n\n*   **Pun:** The core of the joke relies on puns related to the physical characteristics of pineapples and coconu

3.2.3 Final Outcome

In [12]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'Mango', 'joke': 'Why did the mango break up with the apricot?\n\nBecause it said, "I\'m starting to feel like you\'re taking me for GRANITE!" (As in, taking me for granted, but a play on the word "granite" which sounds like the pit of a mango!)', 'explaination': 'Okay, here\'s a breakdown of the mango joke:\n\n**Explanation:**\n\nThe joke is about a mango ending a relationship with an apricot. The reason given is that the mango feels unappreciated. Instead of saying the common phrase "taking me for *granted*," it says "taking me for *granite*." This is funny because:\n\n*   **"Granite" sounds like "granted."**\n*   **"Granite" is a type of rock.**\n*   **Mangoes have a hard, stony pit inside them.** This pit is technically called a "stone" or "seed," but the association with something hard and rock-like is there.\n\nSo, the joke implies that the apricot is treating the mango like a hard, unyielding object instead of a valued partner.\n\n**Wordplay:**\n\n

In [13]:
workflow.get_state(config12)

StateSnapshot(values={'topic': 'Pineapple', 'joke': 'Why did the pineapple break up with the coconut?\n\nBecause he said she was too clingy! He needed some space to, you know, be a little more prickly! ', 'explaination': 'Okay, here\'s the breakdown of the pineapple joke:\n\n**Explanation:**\n\nThe joke is funny because it uses the characteristics of pineapples and coconuts to create a humorous situation about a romantic breakup. It plays on the common stereotype of clingy partners needing space in a relationship.\n\n*   The coconut is described as "clingy" because coconuts grow in clusters on palm trees, appearing to be close together or "clingy".\n*   The pineapple wants space to be "prickly" because pineapples have a tough, spiky exterior. He wants to embrace his natural characteristic, which implies he wants to be more independent and less easy to approach.\n\n**Wordplay/Humor Style:**\n\n*   **Pun:** The core of the joke relies on puns related to the physical characteristics of pi

3.2.4 Chat History by config number

In [14]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Mango', 'joke': 'Why did the mango break up with the apricot?\n\nBecause it said, "I\'m starting to feel like you\'re taking me for GRANITE!" (As in, taking me for granted, but a play on the word "granite" which sounds like the pit of a mango!)', 'explaination': 'Okay, here\'s a breakdown of the mango joke:\n\n**Explanation:**\n\nThe joke is about a mango ending a relationship with an apricot. The reason given is that the mango feels unappreciated. Instead of saying the common phrase "taking me for *granted*," it says "taking me for *granite*." This is funny because:\n\n*   **"Granite" sounds like "granted."**\n*   **"Granite" is a type of rock.**\n*   **Mangoes have a hard, stony pit inside them.** This pit is technically called a "stone" or "seed," but the association with something hard and rock-like is there.\n\nSo, the joke implies that the apricot is treating the mango like a hard, unyielding object instead of a valued partner.\n\n**Wordplay:**\n\

In [15]:
list(workflow.get_state_history(config12))

[StateSnapshot(values={'topic': 'Pineapple', 'joke': 'Why did the pineapple break up with the coconut?\n\nBecause he said she was too clingy! He needed some space to, you know, be a little more prickly! ', 'explaination': 'Okay, here\'s the breakdown of the pineapple joke:\n\n**Explanation:**\n\nThe joke is funny because it uses the characteristics of pineapples and coconuts to create a humorous situation about a romantic breakup. It plays on the common stereotype of clingy partners needing space in a relationship.\n\n*   The coconut is described as "clingy" because coconuts grow in clusters on palm trees, appearing to be close together or "clingy".\n*   The pineapple wants space to be "prickly" because pineapples have a tough, spiky exterior. He wants to embrace his natural characteristic, which implies he wants to be more independent and less easy to approach.\n\n**Wordplay/Humor Style:**\n\n*   **Pun:** The core of the joke relies on puns related to the physical characteristics of p